In [1]:
from TCSPC import *

In [2]:
EGFP = Simulation([0.497,0.503],[2.43,3.07])

In [33]:
tdata = np.loadtxt('EGFPt.csv')
ydata = np.loadtxt('EGFPy2.csv')

In [3]:
def trim_rescale_data(tdata,ydata,end =int(0.6*380)):
    max_idx = np.argmax(ydata) #index of data point with maximum photon count N(0)
    tdata = tdata[:end-max_idx] #start from t = 0
    ydata = ydata[max_idx:end]  #start from max.
    yerr = ydata/ydata[0]*np.sqrt(1/ydata+1/ydata[0]) #error after scaling
    ydata = ydata/ydata[0] # scale y data such that the beginning is 1 
    weights = 1/yerr #weighted by 1/yerr, yerr is error after scaling ydata
    return tdata,ydata

In [4]:
def exp(t, A, tau):
    return A * np.exp(-t/tau)

In [5]:
tdata,ydata = trim_rescale_data(EGFP.t,EGFP.y2)

In [86]:
def poisson_deviance_residual(observed, expected):
    # Ensure no invalid values
    valid = (observed > 0) & (expected > 0)
    residual = np.zeros_like(observed)
    residual[valid] = abs( 2 * (observed[valid] * np.log(observed[valid] / expected[valid]) - (observed[valid] - expected[valid])))
    return np.sum(residual)

def residual(p, t, data):
    v = p.valuesdict()
    generative = v['c'] #constant background
    M = 0
    while f'A{M}' in v:
        generative += exp(t, v[f'A{M}'], v[f'tau{M}'])
        M += 1
    
    return poisson_deviance_residual(data, generative)


In [87]:
def initial_peak_params(M,A_guess,tau_guess):
    p = lmfit.Parameters()
    p.add_many(('c', 0, True, 0, 1)) #constant background
    for i in range(M): #for each component
        p.add_many((f'A{i}', A_guess[i], True,0), #amplitude
                   (f'tau{i}', tau_guess[i], True, 0)) #lifetime
    p[f'A{M-1}'].set(expr = f'1 {"".join([f"- A{i}" for i in range(M-1)])}') #fix the amplitude of last component
    return p



### bi-exp decay fit to EGFP

In [88]:
p1 = initial_peak_params(2,EGFP.amp,EGFP.tau)
mi1 = lmfit.minimize(residual, p1, args=(tdata, ydata), method='powell')

print(lmfit.fit_report(mi1))


[[Fit Statistics]]
    # fitting method   = Powell
    # function evals   = 463
    # data points      = 1
    # variables        = 4
    chi-square         = 1.3288e-05
    reduced chi-square = 1.3288e-05
    Akaike info crit   = -3.22863221
    Bayesian info crit = -11.2286322
##  Warning: uncertainties could not be estimated:
[[Variables]]
    c:     8.1749e-06 (init = 0)
    A0:    0.49539764 (init = 0.497)
    tau0:  2.37295246 (init = 2.43)
    A1:    0.50460236 == '1 - A0'
    tau1:  3.12831460 (init = 3.07)


In [71]:
mi1

In [81]:
result = mi1
del result.params['A1']

In [83]:
info_df,par_df = fit_df([result])


In [84]:
par_df

c  \
0 _val                                                 0.000003   
  init_value                                                  0   
  stderr                                               0.000066   
  correl      {'A0': '-0.466', 'tau0': '-0.455', 'tau1': '-0...   

                                                         A0  \
0 _val                                             0.492887   
  init_value                                          0.497   
  stderr                                           0.296631   
  correl      {'c': '-0.466', 'tau0': '0.999', 'tau1': '1'}   

                                                         tau0  \
0 _val                                               2.370347   
  init_value                                             2.43   
  stderr                                             0.212296   
  correl      {'c': '-0.455', 'A0': '0.999', 'tau1': '0.998'}   

                                                     tau1  
0 _val                                           3.128195  
  init_value                                         3.07  
  stderr                                          0.24093  
  correl      {'c': '-0.481', 'A0': '1', 'tau0': '0.998'}

### mono-exp decay fit to EGFP

In [89]:
p1 = initial_peak_params(1,EGFP.amp,EGFP.tau)
mi1 = lmfit.minimize(residual, p1, args=(tdata, ydata), method='powell')

print(lmfit.fit_report(mi1))


[[Fit Statistics]]
    # fitting method   = Powell
    # function evals   = 148
    # data points      = 1
    # variables        = 2
    chi-square         = 2.7040e-05
    reduced chi-square = 2.7040e-05
    Akaike info crit   = -6.51819207
    Bayesian info crit = -10.5181921
##  Warning: uncertainties could not be estimated:
[[Variables]]
    c:     0.00254199 (init = 0)
    A0:    1.00000000 == '1 '
    tau0:  2.72299181 (init = 2.43)


In [90]:
residual(mi1.params,tdata,ydata)

0.005200003204135892